#### 성근 note

In [3]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from itertools import count
import pandas as pd
from datetime import datetime, timedelta
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np

# import glob
# from pyexcel.cookbook import merge_all_to_a_book

In [4]:
chrome_dir='C:/Yeajin/chromedriver_win32/chromedriver'
save_dir='C:/Yeajin/리얼데이터/마일스톤/crawling/'

pstock_url='http://www.pstock.co.kr/2005pstock/main.asp'
naver_url='https://finance.naver.com/item/main.nhn?code='
url_38='http://www.38.co.kr/html/fund/index.htm?o=nw&page='

df_dtype={'code':str} 

In [5]:
# create initial search list with code

def create_initial_search_list():
    code_list=[]
    name_list=[]

    for p in count(1): # 38에서 페이지# iteration 
        try:
            url=url_38+str(p)
            driver.get(url)
            html=driver.page_source
            soup=BeautifulSoup(html,'html.parser')
            for i in range(1,21): # 한 페이지 내, (max: 20개) 줄 단위 iteration 
                yn=driver.find_element_by_xpath(f'/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[4]/tbody/tr[2]/td/table/tbody/tr[{i}]/td[9]').text
                if  yn.strip()!='예정': # [신규상장]탭 > '첫날종가(원)' != '예정' 인 경우 스크랩 
                    name=driver.find_element_by_xpath(f'/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[4]/tbody/tr[2]/td/table/tbody/tr[{i}]/td[1]/a')
                    name_list.append(name.text)
                    name.send_keys(Keys.CONTROL + '\n') # '새 창으로 보기'
                    driver.switch_to.window(driver.window_handles[1]) # '새 창으로 탭 전환'
                    code=driver.find_element_by_xpath('/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[2]/tbody/tr[2]/td[4]').text
                    code_list.append(code.strip())
                    driver.close() # 켜진 '새 창' 닫기
                    driver.switch_to.window(driver.window_handles[0])
        except:
            print(name_list)
            break

    # create code df
    mCols=[]
    code_df=pd.DataFrame(mCols)

    code_df['name']=name_list
    code_df['code']=code_list

    today=datetime.today().strftime('%Y%m%d')

#     code_df.to_csv(f'기업리스트트.csv',encoding='utf-8', index=False)
    code_df.to_csv(f'기업리스트.csv',encoding='utf-8')    # (수정1020) 오타수정 
    
    return name_list, code_list

In [14]:
# create initial search list

# def create_pstock_initial_search_list():
def create_pstock_initial_search_list(name_list, ):
    
    new_stocks_1=[]
    new_stocks_2=[]
    new_stocks=name_list
    for index,new_name in enumerate(new_stocks):
        if '(유가' in new_name or '(구' in new_name:
            new_name=new_name.split('(')[0]
            new_stocks_1.append(new_name)
        elif '스팩' in new_name and '호' in new_name:
            new_name=new_name.split('스팩')
            new_stocks_2.append(new_name)
        else:
            new_stocks_1.append(new_name)
    new_stocksss=new_stocks_1+new_stocks_2
    
    return new_stocksss, new_stocks_1, new_stocks_2

def create_naver_initial_search_list():
    code_df=pd.read_csv('기업리스트.csv',encoding='utf-8',index_col=[0], dtype=df_dtype)
    naver_code_list=code_df['code'].tolist()
    naver_name_list=code_df['name'].tolist() # == name_list ( in ln 8)
    
    return naver_name_list, naver_code_list

In [7]:
# 검색 함수

# def pstock_search():
def pstock_search(pstock_url, new_stocks_1, new_stocks_2):
    driver.get(pstock_url)
    p_search=driver.find_element_by_xpath('//*[@id="head_main"]')
    p_search.clear()
# ===============================================================
# new_stocks_1 : 기업명 // new_stocks_2 : '스팩' 포함 기업명
# ===============================================================

    if new_name in new_stocks_1:
        p_search.send_keys(new_name)
        driver.find_element_by_xpath('//*[@id="up_win"]/table[2]/tbody/tr[1]/td[2]/table/tbody/tr/td[3]/table/tbody/tr[2]/td/table/tbody/tr/td[4]/input').click()
# =============================================================================
# 분리해서 검색하고, [기업명,ㅁ호] 중 기업명을 검색하여 ㅁ호를 포함한 이름을 선택
# =============================================================================

    elif new_name in new_stocks_2:
        p_search.send_keys(new_name[0])    
        driver.find_element_by_xpath('//*[@id="up_win"]/table[2]/tbody/tr[1]/td[2]/table/tbody/tr/td[3]/table/tbody/tr[2]/td/table/tbody/tr/td[4]/input').click()
        html=driver.page_source
        i=2
        while True:
            name=driver.find_element_by_xpath(f'/html/body/center/table/tbody/tr[2]/td[2]/table[2]/tbody/tr[3]/td[2]/table[2]/tbody/tr[{i}]/td[1]/a').text
            if new_name[1] in name:
                company=driver.find_element_by_xpath(f'/html/body/center/table/tbody/tr[2]/td[2]/table[2]/tbody/tr[3]/td[2]/table[2]/tbody/tr[{i}]/td[1]/a').click()
                break
            else:
                i+=1

In [8]:
def naver_search():
    """
    'searched.append(n)'
    : '검색기업' 결과가 나오지 않은 경우, 'n'=='검색되지 못한 기업이름'을 append 후, 에러시 출력에 사용
    """
    searched=[]
    driver.get(naver_url+str(code))
    n=driver.find_element_by_xpath('//*[@id="middle"]/div[1]/div[1]/h2/a').text
    searched.append(n)

In [9]:
# 파싱 함수
def pstock_parsing():
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    tbodys=soup.find_all('tbody')
    for i in range(len(tbodys)-1,0,-1):
        if soup.find_all('tbody')[i].find_all('tr')[-1].find_all('td')[0].text=='상장예정 주식수':
            circulation_stocks=soup.find_all('tbody')[i].find_all('tr')[-3].find_all('td')[1].text.strip() # 유통가능주식수 
            circulation_share=soup.find_all('tbody')[i].find_all('tr')[-3].find_all('td')[2].text.strip() # 유통가능지분율
            new_listing_stocks=soup.find_all('tbody')[i].find_all('tr')[-1].find_all('td')[1].text.strip() # 상장예정주식수
            new_listing_share=soup.find_all('tbody')[i].find_all('tr')[-1].find_all('td')[2].text.strip() # 상장예정지분율
    
    for j in range(len(tbodys)):
        try:
            if soup.find_all('tbody')[j].find_all('tr')[6].find_all('td')[0].text=='확정공모가':
                offering_price=soup.find_all('tbody')[j].find_all('tr')[6].find_all('td')[1].text[:-5]
        except:
            continue 
    
    circulation_price=int(re.sub('[,]','',circulation_stocks))*int(re.sub('[,]','',offering_price)) # 총유통가능금액 = 확정공모가 * 유통가능주식수
    tot_price=int(re.sub('[,]','',new_listing_stocks))*int(re.sub('[,]','',offering_price)) # 시가총액 = 확정공모가 * 상장예정주식수
    
    circulation_stocks_list.append(circulation_stocks)
    circulation_share_list.append(circulation_share)
    circulation_price_list.append(circulation_price)
    new_listing_stocks_list.append(new_listing_stocks)
    new_listing_share_list.append(new_listing_share)
    tot_price_list.append(tot_price)

In [10]:
def naver_parsing():
    tmp_price_list=[]
    tmp_date_list=[]
    
    driver.get(naver_url+str(code)) # '네이버 금융' 기본 url + 공모주식코드
    #== 첫 화면 '종목명' 확인용 
    print(driver.find_element_by_xpath('//*[@id="middle"]/div[1]/div[1]/h2/a').text)
    #==

    driver.find_element_by_xpath('//*[@id="content"]/ul/li[2]/a').click() # 시세 탭 클릭 
    iframes=driver.find_elements_by_tag_name('iframe') # *** iframe이 다수 
    driver.switch_to.frame(iframes[-2]) # <<< iframe 중 '일별시세'의 위치가 뒤에서 2번쨰

    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    pages=soup.find('table',class_='Nnavi').find_all('td') # 페이지 네비 칸 전체 파싱 

    end_num=0
    if len(pages)!=1: # 1개 페이지 이상인 경우 >>> 
        driver.find_element_by_class_name('pgRR').click() # 페이지 네비 '맨뒤' 클릭
        cell_nums=[3,4,5,6,7,11,12,13,14,15] # 일별시세 내 '맨뒤' 내용 10개에 해당하는 idx 칸 번호
        try:
            for i in cell_nums: # 첫번쨰 칸 부터 훑어 내려와서 
                price=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[7]/span').text # <<< (위치001)
                #date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[1]/span').text
                prev_i=i
                tmp_price_list.append(price) # 가격 정보를 쭉 - append 하고 
                #tmp_date_list.append(date)
        except: # try에서 막힌 경우 >>> '가격'정보가 없어서 파싱 예외처리 나면 (위치001), 내려와서
            first_price=tmp_price_list[-1] # 마지막 가격 정보 == 첫날 종가 
            first_date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{prev_i}]/td[1]/span').text # 상장첫날 날짜
            first_price_list.append(first_price) 
            print(first_price)
    else: #  페이지가 1개 뿐인 경우 >>> 바로 목록내 컨텐츠로 
        cell_nums=[3,4,5,6,7,11,12,13,14,15]
        try:
            for i in cell_nums:
                price=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[7]/span').text
#                 date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{i}]/td[1]/span').text
                prev_i=i
                tmp_price_list.append(price)
        except:
            first_price=tmp_price_list[-1]
            first_date=driver.find_element_by_xpath(f'/html/body/table[1]/tbody/tr[{prev_i}]/td[1]/span').text
            first_price_list.append(first_price)
            print(first_price)
    
    
    n_dates=[]
    date=datetime.strptime(first_date,'%Y.%m.%d')
    for d in range(3):
        date=date+timedelta(weeks=4)    
        n_date=date.strftime('%Y.%m.%d')
        n_dates.append(n_date)
        print(n_date)
    
    return first_price_list, first_date_list

In [11]:
# create df
def create_df():
    mCols=[]
    p_df=pd.DataFrame(mCols)
    
    p_df['기업명']=search_list
    p_df['유통가능주식수']=circulation_stocks_list
    p_df['유통가능지분율']=circulation_share_list
    p_df['유통가능금액']=circulation_price_list
    p_df['상장예정주식수']=new_listing_stocks_list
    p_df['상장예정지분율']=new_listing_share_list
    p_df['시가총액']=tot_price_list
    p_df['상장첫날거래량']=first_price_list
    
    today=datetime.today().strftime('%Y%m%d')
    
#     today_note=open('date_info.txt','w',encoding='utf-8')
#     print(f'{today}',file=today_note)
#     today_note.close()
    
    p_df.to_csv('pstock_naver.csv',encoding='utf-8')
    p_df.to_csv(save_dir+f'pstock_naver_{today}.csv',encoding='utf-8')
    

In [12]:
# create failed note
def failed_note():    
    today=datetime.today().strftime('%y%m%d')
    failed_note=open(save_dir+f'failed/failed_note_{today}.txt','w',encoding='utf-8')
    print(f'pstock\n 검색불가 : {pstock_search_failed} \n파싱불가 : {pstock_parsing_failed}', file=failed_note)
    print(f'\n naver \n 검색불가 : {naver_search_failed} \n파싱불가 : {naver_parsing_failed}', file=failed_note)
    
    failed_note.close()
    

In [13]:
search_list=[]

pstock_search_failed=[]
pstock_parsing_failed=[]

naver_search_failed=[]
naver_parsing_failed=[]

company_name_list=[]

circulation_stocks_list=[]
circulation_share_list=[]
circulation_price_list=[]

new_listing_stocks_list=[]
new_listing_share_list=[]
tot_price_list=[]

first_price_list=[]
first_date_list=[]

In [54]:
# ================================================================
# 크롤링 수행    -    가장 처음 시작시 수행
# ================================================================

driver=webdriver.Chrome(chrome_dir)
# 1.0.1
name_list,code_list=create_initial_search_list()

# 2.p.1
new_stocksss,new_stocks_1,new_stocks_2=create_pstock_initial_search_list()
# 2.n.1
naver_name_list,naver_code_list=create_naver_initial_search_list()

# pstock

for new_name in new_stocksss:
    try:
        pstock_search()     # search 함수 실행 # # 2.p.2
        name=driver.find_element_by_xpath('/html/body/center/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td[1]/table[6]/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/b').text.strip()
        search_list.append(name)
    except:
        search_list.append(np.nan)
        pstock_search_failed.append(new_name)
        circulation_stocks_list.append(np.nan)
        circulation_share_list.append(np.nan)
        circulation_price_list.append(np.nan)
        new_listing_stocks_list.append(np.nan)
        new_listing_share_list.append(np.nan)
        tot_price_list.append(np.nan)
        continue
        
    try:
        pstock_parsing()     # # 2.p.3
        company_name_list.append(new_name)
    except:
        pstock_parsing_failed.append(new_name)
        circulation_stocks_list.append(np.nan)
        circulation_share_list.append(np.nan)
        circulation_price_list.append(np.nan)
        new_listing_stocks_list.append(np.nan)
        new_listing_share_list.append(np.nan)
        tot_price_list.append(np.nan)        
        continue
        
# naver

naver_dict=dict(zip(naver_code_list,naver_name_list))

# for code in naver_code_list[:30]:    # <<< (1020)
for code in naver_code_list:
    try:
        naver_search()        # # 2.n.2
    except:
        s_failed_name=naver_dict.get(code)
        naver_search_failed.append(s_failed_name)
        first_price_list.append(np.nan)
        continue
    
    try:
        naver_parsing()      # # 2.n.3
    except:
        p_failed_name=naver_dict.get(code)
        naver_parsing_failed.append(p_failed_name)
        first_price_list.append(np.nan)
        continue
        
        
create_df()        # 3.1.1
failed_note()      # 3.1.2
        
    
print(f'****** Pstock Search Failed : {pstock_search_failed}')
print(f'****** Pstock Parsing Failed: {pstock_parsing_failed}')
print(f'****** Naver Search Failed : {naver_search_failed}')
print(f'****** Naver Parsing Failed: {naver_parsing_failed}')

피플바이오
423,111
2020.11.16
2020.12.14
2021.01.11
빅히트
423,111
2020.11.12
2020.12.10
2021.01.07
넥스틴
423,111
2020.11.05
2020.12.03
2020.12.31
원방테크
1,530,069
2020.10.22
2020.11.19
2020.12.17
엔에이치스팩17호
551,120
2020.10.21
2020.11.18
2020.12.16
비나텍
0
2013.07.29
2013.08.26
2013.09.23
박셀바이오
4,090,940
2020.10.20
2020.11.17
2020.12.15
비비씨
2,415,082
2020.10.19
2020.11.16
2020.12.14
교보10호스팩
227,158
2020.10.16
2020.11.13
2020.12.11
핌스
5,871,649
2020.10.16
2020.11.13
2020.12.11
압타머사이언스
이오플로우
6,672,023
2020.10.12
2020.11.09
2020.12.07
카카오게임즈
561,750
2020.10.08
2020.11.05
2020.12.03
피엔케이피부임상연구센타
8,304,697
2020.10.07
2020.11.04
2020.12.02
코람코에너지리츠
2,881,153
2020.09.28
2020.10.26
2020.11.23
미래에셋대우스팩 5호
459,818
2020.09.25
2020.10.23
2020.11.20
아이디피
9,251,446
2020.09.21
2020.10.19
2020.11.16
셀레믹스
11,903,741
2020.09.18
2020.10.16
2020.11.13
미투젠
5,518,960
2020.09.15
2020.10.13
2020.11.10
브랜드엑스코퍼레이션
4,135,892
2020.09.10
2020.10.08
2020.11.05
영림원소프트랩
6,132,668
2020.09.09
2020.10.07
2020.11.04
한국파마
14,406,068
202